In [ ]:
# default_exp diffquant_utils

# Paths

In [ ]:
#export
import os
import pathlib
if "__file__" in globals():#only run in the translated python file, as __file__ is not defined with ipython
    INTABLE_CONFIG = os.path.join(pathlib.Path(__file__).parent.absolute(), "configs", "intable_config.yaml") #the yaml config is located one directory below the python library files


# Utility Functions

## Reformatting and transformation functions

In [ ]:
#export

def get_samples_used_from_samplemap_file(samplemap_file, cond1, cond2):
    samplemap_df = load_samplemap(samplemap_file)
    return get_samples_used_from_samplemap_df(samplemap_df, cond1, cond2)


def get_samples_used_from_samplemap_df(samplemap_df, cond1, cond2):
    samples_c1 = samplemap_df[[cond1 == x for x in samplemap_df["condition"]]]["sample"] #subset the df to the condition
    samples_c2 = samplemap_df[[cond2 == x for x in samplemap_df["condition"]]]["sample"]
    return list(samples_c1), list(samples_c2)

def get_all_samples_from_samplemap_df(samplemap_df):
    return list(samplemap_df["sample"])

In [ ]:
#export
import pandas as pd

def get_samplenames_from_input_df(data):
    """extracts the names of the samples of the AQ input dataframe"""
    names = list(data.columns)
    names.remove('protein')
    names.remove('ion')
    return names

In [ ]:
#export
import numpy as np
def filter_df_to_minrep(quant_df_wideformat, samples_c1, samples_c2, minrep):
    """filters dataframe in alphaquant format such that each column has a minimum number of replicates
    """
    quant_df_wideformat = quant_df_wideformat.replace(0, np.nan)
    df_c1_minrep = quant_df_wideformat[samples_c1].dropna(thresh = minrep, axis = 0)
    df_c2_minrep = quant_df_wideformat[samples_c2].dropna(thresh = minrep, axis = 0)
    idxs_both = df_c1_minrep.index.intersection(df_c2_minrep.index)
    quant_df_reduced = quant_df_wideformat.iloc[idxs_both].reset_index()
    return quant_df_reduced


In [ ]:
#export
def get_condpairname(condpair):
    return f"{condpair[0]}_VS_{condpair[1]}"

In [ ]:
#export

def get_quality_score_column(acquisition_info_df):
    if "FG.ShapeQualityScore" in acquisition_info_df.columns:
        param = "FG.ShapeQualityScore"
    elif "Quantity.Quality" in acquisition_info_df.columns:
        param = "Quantity.Quality"
    return param

In [ ]:
#export
import os

def make_dir_w_existcheck(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

In [ ]:
#export
import os
def get_results_plot_dir_condpair(results_dir, condpair):
    results_dir_plots = f"{results_dir}/{condpair}_plots"
    make_dir_w_existcheck(results_dir_plots)
    return results_dir_plots

In [ ]:
#export
def get_middle_elem(sorted_list):
    nvals = len(sorted_list)
    if nvals==1:
        return sorted_list[0]
    middle_idx = nvals//2
    if nvals%2==1:
        return sorted_list[middle_idx]
    return 0.5* (sorted_list[middle_idx] + sorted_list[middle_idx-1])

In [ ]:
#export
import numpy as np
def get_nonna_array(array_w_nas):
    res = []
    isnan_arr = np.isnan(array_w_nas)

    for idx in range(len(array_w_nas)):
        sub_res = []
        sub_array = array_w_nas[idx]
        na_array = isnan_arr[idx]
        for idx2 in range(len(sub_array)):
            if not na_array[idx2]:
               sub_res.append(sub_array[idx2])
        res.append(np.array(sub_res))
    return np.array(res)

In [ ]:
#export
import numpy as np
def get_non_nas_from_pd_df(df):
    return {
        pep_name: sub_vals[~np.isnan(sub_vals)] for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }

In [ ]:
#export
import numpy as np
def get_ionints_from_pd_df(df):
    return {
        pep_name: sub_vals for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }

In [ ]:
#export
def invert_dictionary(my_map):
    inv_map = {}
    for k, v in my_map.items():
        inv_map[v] = inv_map.get(v, []) + [k]
    return inv_map

In [ ]:
#export
import statistics

def get_z_from_p_empirical(p_emp,p2z):
    p_rounded = np.format_float_scientific(p_emp, 1)
    if p_rounded in p2z:
        return p2z.get(p_rounded)
    z = statistics.NormalDist().inv_cdf(float(p_rounded))
    p2z[p_rounded] = z
    return z

In [ ]:
#export
import anytree
def get_levelnodes_from_nodeslist(nodeslist, level):
    levelnodes = []
    for node in nodeslist:
        precursors = anytree.findall(node, filter_= lambda x : (x.type == level))
        levelnodes.extend(precursors)
    return levelnodes

In [ ]:
#export
def count_fraction_outliers_from_expected_fc(result_df, threshold, expected_log2fc):
    num_outliers = sum([abs(x-expected_log2fc)> threshold for x in result_df["log2fc"]])
    fraction_outliers = num_outliers/len(result_df["log2fc"])
    print(f"{round(fraction_outliers, 2)} outliers")
    return fraction_outliers

In [ ]:
#export
import anytree

def find_node_parent_at_level(node, level):
    if node.type == level:
        return node
    while node.parent is not None:
        node = node.parent
        if node.type == level:
            return node
        

In [ ]:
#hide
from anytree import Node

def test_find_node_parent_at_level():

    udo = Node("Udo", type = 'granddad')
    marc = Node("Marc", parent=udo, type = 'dad')
    lian = Node("Lian", parent=marc, type = 'base')
    dan = Node("Dan", parent=udo, type ='dad')
    jet = Node("Jet", parent=dan, type ='base')
    jan = Node("Jan", parent=dan, type ='base')
    joe = Node("Joe", parent=dan, type ='base')

    assert find_node_parent_at_level(lian, 'dad').name == 'Marc'
    assert find_node_parent_at_level(lian, 'granddad').name == 'Udo'
    assert find_node_parent_at_level(jet, 'dad').name == 'Dan'

test_find_node_parent_at_level()


In [ ]:
#export
import anytree

def check_if_node_is_included(node):
    while node.type != "gene":
        if node.cluster != 0:
            return False
        node = node.parent
    
    return True
        


In [ ]:
#hide
from anytree import Node
from anytree import RenderTree


def test_check_if_node_is_included():

    udo = Node("Udo", type = 'gene', cluster =-1)
    marc = Node("Marc", parent=udo, type = 'dad', cluster = 0)
    lian = Node("Lian", parent=marc, type = 'base', cluster = 0)
    dan = Node("Dan", parent=udo, type ='dad', cluster = 1)
    jet = Node("Jet", parent=dan, type ='base', cluster = 0)
    jan = Node("Jan", parent=dan, type ='base', cluster = 0)
    joe = Node("Joe", parent=dan, type ='base', cluster = 0)
    print(RenderTree(udo))
    assert check_if_node_is_included(jet) == False
    assert check_if_node_is_included(lian) == True
    assert check_if_node_is_included(dan) == False
    assert check_if_node_is_included(marc) == True

test_check_if_node_is_included()


## I/O functions

In [ ]:
#export
def write_chunk_to_file(chunk, filepath ,write_header):
    """write chunk of pandas dataframe to a file"""
    chunk.to_csv(filepath, header=write_header, mode='a', sep = "\t", index = None)

## Documentation functions

In [ ]:
#export
import logging
import os
import sys


def set_logger(
    *,log_path = None,
    log_file_name="",
    stream: bool = True,
    log_level: int = logging.INFO,
    overwrite: bool = False,
) -> str:
    """Set the log stream and file.
    All previously set handlers will be disabled with this command.
    Parameters
    ----------
    log_file_name : str, None
        The file name to where the log is written.
        Folders are automatically created if needed.
        This is relative to the current path. When an empty string is provided,
        a log is written to the AlphaTims "logs" folder with the name
        "log_yymmddhhmmss" (reversed timestamp year to seconds).
        If None, no log file is saved.
        Default is "".
    stream : bool
        If False, no log data is sent to stream.
        If True, all logging can be tracked with stdout stream.
        Default is True.
    log_level : int
        The logging level. Usable values are defined in Python's "logging"
        module.
        Default is logging.INFO.
    overwrite : bool
        If True, overwrite the log_file if one exists.
        If False, append to this log file.
        Default is False.
    Returns
    -------
    : str
        The file name to where the log is written.
    """
    import time
    global PROGRESS_CALLBACK
    log_path = os.path.join(BASE_PATH, "logs")
    root = logging.getLogger()
    formatter = logging.Formatter(
        '%(asctime)s> %(message)s', "%Y-%m-%d %H:%M:%S"
    )
    root.setLevel(log_level)
    while root.hasHandlers():
        root.removeHandler(root.handlers[0])
    if stream:
        stream_handler = logging.StreamHandler(sys.stdout)
        stream_handler.setLevel(log_level)
        stream_handler.setFormatter(formatter)
        root.addHandler(stream_handler)
    if log_file_name is not None:
        if log_file_name == "":
            if not os.path.exists(log_path):
                os.makedirs(log_path)
            log_file_name = log_path
        log_file_name = os.path.abspath(log_file_name)
        if os.path.isdir(log_file_name):
            current_time = time.localtime()
            current_time = "".join(
                [
                    f'{current_time.tm_year:04}',
                    f'{current_time.tm_mon:02}',
                    f'{current_time.tm_mday:02}',
                    f'{current_time.tm_hour:02}',
                    f'{current_time.tm_min:02}',
                    f'{current_time.tm_sec:02}',
                ]
            )
            log_file_name = os.path.join(
                log_file_name,
                f"log_{current_time}.txt"
            )
        directory = os.path.dirname(log_file_name)
        if not os.path.exists(directory):
            os.makedirs(directory)
        if overwrite:
            file_handler = logging.FileHandler(log_file_name, mode="w")
        else:
            file_handler = logging.FileHandler(log_file_name, mode="a")
        file_handler.setLevel(log_level)
        file_handler.setFormatter(formatter)
        root.addHandler(file_handler)
    return log_file_name

In [ ]:
#export
import yaml
import os.path
import pathlib
import re

def load_method_parameters(results_dir):
    params_file = f"{results_dir}/aq_parameters.yaml"
    return load_config(params_file)

def store_method_parameters(local_vars_dict, results_dir):
    method_params = get_methods_dict_from_local_vars(local_vars_dict)
    add_ml_input_file_location(method_params)
    params_file = f"{results_dir}/aq_parameters.yaml"
    if os.path.exists(params_file):
        previous_params = load_method_parameters(results_dir)
        previous_params = {x:y for x, y in previous_params.items() if x not in method_params.keys()}
        method_params.update(previous_params)
    if not os.path.exists(f"{results_dir}/"):
        os.makedirs(f"{results_dir}/")
    save_dict_as_yaml(method_params, params_file)

def save_dict_as_yaml(dict, file):
    with open(file, 'w') as outfile:
        yaml.dump(dict, outfile, default_flow_style=False)


def get_methods_dict_from_local_vars(local_vars):
    method_params = {}
    for x in local_vars.keys():
        if local_vars[x] is None:
            continue
        if (("_df" not in x) and ('condpair' not in x) and ('sys'!=x) and ('runconfig' != x)):
            if ("input_file" in x) or ("results_dir" in x):
                method_params[x] = os.path.abspath(local_vars[x])
            else:
                method_params[x] = local_vars[x]
    return method_params

def add_ml_input_file_location(method_params):
    input_file = method_params.get("input_file")
    if ".aq_reformat" in input_file:
        ml_input_file = get_path_to_unformatted_file(input_file)
    else:
        ml_input_file = input_file
    method_params["ml_input_file"] = ml_input_file

def get_path_to_unformatted_file(input_file_name):
    prefixes = input_file_name.split(".")[:-3]
    cleaned_filename = ".".join(prefixes)
    return cleaned_filename


# Input Parsers
The Alphaquant pipeline is run using a generic wide-table input format, as specified in the documentation. The following parsers convert long format tables as provided e.g. by Spectronaut or DIA-NN into this generic format. The configuration for the parsers is set by a yaml file.

## Convert long format to wide format

### Parse .yaml file
The relevant parameters for reading and reformatting the long table are stored in the "intable_config.yaml" file. The functions below are for reading and reformating the config info

In [ ]:
#export
import yaml
import itertools

def get_relevant_columns(protein_cols, ion_cols, sample_ID, quant_ID, filter_dict):
    filtcols = []
    for filtconf in filter_dict.values():
        filtcols.append(filtconf.get('param'))
    relevant_cols = protein_cols + ion_cols + [sample_ID] + [quant_ID] + filtcols
    relevant_cols = list(set(relevant_cols)) # to remove possible redudancies
    return relevant_cols


def get_relevant_columns_config_dict(config_typedict):
    filtcols = []
    dict_ioncols = []
    for filtconf in config_typedict.get('filters', {}).values():
        filtcols.append(filtconf.get('param'))

    if 'ion_hierarchy' in config_typedict.keys():
        for headr in config_typedict.get('ion_hierarchy').values():
            ioncols = list(itertools.chain.from_iterable(headr.get("mapping").values()))
            dict_ioncols.extend(ioncols)
    quantID = config_typedict.get("quant_ID")
    if type(quantID) ==type("string"):
        quant_ids = [config_typedict.get("quant_ID")]
    elif quantID == None:
        quant_ids = []
    else:
        quant_ids = list(config_typedict.get("quant_ID").values())
    relevant_cols = config_typedict.get("protein_cols") + config_typedict.get("ion_cols", []) + [config_typedict.get("sample_ID")] + quant_ids + filtcols + dict_ioncols
    relevant_cols = list(set(relevant_cols)) # to remove possible redudancies
    return relevant_cols



def get_config_columns(config_dict):
    protein_cols = config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    sample_ID = config_dict.get("sample_ID")
    quant_ID = config_dict.get("quant_ID")
    filter_dict = config_dict.get("filters", {})
    relevant_cols = get_relevant_columns(protein_cols, ion_cols, sample_ID, quant_ID, filter_dict)
    return relevant_cols, protein_cols, ion_cols, sample_ID, quant_ID, filter_dict


def load_config(config_yaml):
    stream = open(config_yaml, 'r')
    config_all = yaml.safe_load(stream)
    return config_all

def get_type2relevant_cols(config_all):
    type2relcols = {}
    for type in config_all.keys():
        config_typedict = config_all.get(type)
        relevant_cols = get_relevant_columns_config_dict(config_typedict)
        type2relcols[type] = relevant_cols
    return type2relcols

### Filter and reformat

In [ ]:
#export

def filter_input(filter_dict, input):
    for filtname,filterconf in filter_dict.items():
        param = filterconf.get('param')
        comparator = filterconf.get('comparator')
        value = filterconf.get('value')

        if comparator not in [">",">=", "<", "<=", "==", "!="]:
            raise TypeError(f"cannot identify the filter comparator of {filtname} given in the longtable config yaml!")

        if comparator=="==":
            input = input[input[param] ==value]
            continue
        try:
            input = input.astype({f"{param}" : "float"})
        except:
            pass

        if comparator==">":
            input = input[input[param].astype(type(value)) >value]

        if comparator==">=":
            input = input[input[param].astype(type(value)) >=value]

        if comparator=="<":
            input = input[input[param].astype(type(value)) <value]

        if comparator=="<=":
            input = input[input[param].astype(type(value)) <=value]

        if comparator=="!=":
            input = input[input[param].astype(type(value)) !=value]

    return input

In [ ]:
#export
def merge_protein_and_ion_cols(input_df, config_dict):
    protein_cols =  config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    input_df['protein'] = input_df.loc[:, protein_cols].astype('string').sum(axis=1)
    input_df['ion'] = input_df.loc[:, ion_cols].astype('string').sum(axis=1)
    input_df = input_df.rename(columns = {config_dict.get('quant_ID') : "quant_val"})
    return input_df

In [ ]:
#export
import copy
def merge_protein_cols_and_ion_dict(input_df, config_dict):
    """[summary]

    Args:
        input_df ([pandas dataframe]): longtable containing peptide intensity data
        confid_dict ([dict[String[]]]): nested dict containing the parse information. derived from yaml file

    Returns:
        pandas dataframe: longtable with newly assigned "protein" and "ion" columns
    """
    protein_cols = config_dict.get("protein_cols")
    ion_hierarchy = config_dict.get("ion_hierarchy")
    splitcol2sep = config_dict.get('split_cols')
    quant_id_dict = config_dict.get('quant_ID')

    ion_dfs = []
    input_df['protein'] = input_df.loc[:, protein_cols].astype('string').sum(axis=1)

    input_df = input_df.drop(columns = [x for x in protein_cols if x!='protein'])
    for hierarchy_type in ion_hierarchy.keys():
        df_subset = input_df.copy()
        ion_hierarchy_local = ion_hierarchy.get(hierarchy_type).get("order")
        ion_headers_merged, ion_headers_grouped = get_ionname_columns(ion_hierarchy.get(hierarchy_type).get("mapping"), ion_hierarchy_local) #ion headers merged is just a helper to select all relevant rows, ionheaders grouped contains the sets of ionstrings to be merged into a list eg [[SEQ, MOD], [CH]]
        quant_columns = get_quantitative_columns(df_subset, hierarchy_type, config_dict, ion_headers_merged)
        headers = list(set(ion_headers_merged + quant_columns + ['protein']))
        if "sample_ID" in config_dict.keys():
            headers+=[config_dict.get("sample_ID")]
        df_subset = df_subset[headers].drop_duplicates()

        if splitcol2sep is not None:
            if quant_columns[0] in splitcol2sep.keys(): #in the case that quantitative values are stored grouped in one column (e.g. msiso1,msiso2,msiso3, etc.), reformat accordingly
                df_subset = split_extend_df(df_subset, splitcol2sep)
            ion_headers_grouped = adapt_headers_on_extended_df(ion_headers_grouped, splitcol2sep)

        #df_subset = df_subset.set_index(quant_columns)

        df_subset = add_merged_ionnames(df_subset, ion_hierarchy_local, ion_headers_grouped, quant_id_dict, hierarchy_type)
        ion_dfs.append(df_subset)
    input_df = pd.concat(ion_dfs, ignore_index=True)
    return input_df


def get_quantitative_columns(input_df, hierarchy_type, config_dict, ion_headers_merged):
    naming_columns = ion_headers_merged + ['protein']
    if config_dict.get("format") == 'longtable':
        quantcol = config_dict.get("quant_ID").get(hierarchy_type)
        return [quantcol]

    if config_dict.get("format") == 'widetable':
        quantcolumn_candidates = [x for x in input_df.columns if x not in naming_columns]
        if "quant_prefix" in config_dict.keys():
            return [x for x in quantcolumn_candidates if x.startswith(config_dict.get("quant_prefix"))] # in the case that the quantitative columns have a prefix (like "Intensity " in MQ peptides.txt), only columns with the prefix are filtered
        else:
            return quantcolumn_candidates #in this case, we assume that all non-ionname/proteinname columns are quantitative columns


def get_ionname_columns(ion_dict, ion_hierarchy_local):
    ion_headers_merged = []
    ion_headers_grouped = []
    for lvl in ion_hierarchy_local:
        vals = ion_dict.get(lvl)
        ion_headers_merged.extend(vals)
        ion_headers_grouped.append(vals)
    return ion_headers_merged, ion_headers_grouped


def adapt_headers_on_extended_df(ion_headers_grouped, splitcol2sep):
    #in the case that one column has been split, we need to designate the "naming" column
    ion_headers_grouped_copy = copy.deepcopy(ion_headers_grouped)
    for vals in ion_headers_grouped_copy:
        if splitcol2sep is not None:
            for idx in range(len(vals)):
                if vals[idx] in splitcol2sep.keys():
                    vals[idx] = vals[idx] + "_idxs"
    return ion_headers_grouped_copy

def split_extend_df(input_df, splitcol2sep, value_threshold=10):
    """reformats data that is stored in a condensed way in a single column. For example isotope1_intensity;isotope2_intensity etc. in Spectronaut

    Args:
        input_df ([type]): [description]
        splitcol2sep ([type]): [description]
        value_threshold([type]): [description]

    Returns:
        Pandas Dataframe: Pandas dataframe with the condensed items expanded to long format
    """
    if splitcol2sep==None:
        return input_df

    for split_col, separator in splitcol2sep.items():
        idx_name = f"{split_col}_idxs"
        split_col_series = input_df[split_col].str.split(separator)
        input_df = input_df.drop(columns = [split_col])

        input_df[idx_name] = [list(range(len(x))) for x in split_col_series]
        exploded_input = input_df.explode(idx_name)
        exploded_split_col_series = split_col_series.explode()

        exploded_input[split_col] = exploded_split_col_series.replace('', 0) #the column with the intensities has to come after to column with the idxs

        exploded_input = exploded_input.astype({split_col: float})
        exploded_input = exploded_input[exploded_input[split_col]>value_threshold]
        #exploded_input = exploded_input.rename(columns = {'var1': split_col})
    return exploded_input



def add_merged_ionnames(df_subset, ion_hierarchy_local, ion_headers_grouped, quant_id_dict, hierarchy_type):
    """puts together the hierarchical ion names as a column in a given input dataframe"""
    all_ion_headers = list(itertools.chain.from_iterable(ion_headers_grouped))
    columns_to_index = [x for x in df_subset.columns if x not in all_ion_headers]
    df_subset = df_subset.set_index(columns_to_index)

    rows = df_subset[all_ion_headers].to_numpy()
    ions = []

    for row in rows: #iterate through dataframe
        count = 0
        ionstring = ""
        for lvl_idx in range(len(ion_hierarchy_local)):
            ionstring += f"{ion_hierarchy_local[lvl_idx]}"
            for sublvl in ion_headers_grouped[lvl_idx]:
                ionstring+= f"_{row[count]}_"
                count+=1
        ions.append(ionstring)
    df_subset['ion'] = ions
    df_subset = df_subset.reset_index()
    if quant_id_dict!= None:
        df_subset = df_subset.rename(columns = {quant_id_dict.get(hierarchy_type) : "quant_val"})
    return df_subset

In [ ]:
#export
import os.path
def reformat_and_write_longtable_according_to_config(input_file, outfile_name, config_dict, sep = "\t",decimal = ".", enforce_largefile_processing = False, chunksize =1000_000):
    """Reshape a long format proteomics results table (e.g. Spectronaut or DIA-NN) to a wide format table.
    :param file input_file: long format proteomic results table
    :param string input_type: the configuration key stored in the config file (e.g. "diann_precursor")
    """
    filesize = os.path.getsize(input_file)/(1024**3) #size in gigabyte
    file_is_large = (filesize>10 and str(input_file).endswith(".zip")) or filesize>50 or enforce_largefile_processing

    if file_is_large:
        tmpfile_large = f"{input_file}.tmp.longformat.columnfilt.tsv" #only needed when file is large
        #remove potential leftovers from previous processings
        if os.path.exists(tmpfile_large):
            os.remove(tmpfile_large)
        if os.path.exists(outfile_name):
            os.remove(outfile_name)
    
    relevant_cols = get_relevant_columns_config_dict(config_dict)
    input_df_it = pd.read_csv(input_file, sep = sep, decimal=decimal, usecols = relevant_cols, encoding ='latin1', chunksize = chunksize)
    input_df_list = []
    header = True
    for input_df_subset in input_df_it:
        input_df_subset = adapt_subtable(input_df_subset, config_dict)
        if file_is_large:
            write_chunk_to_file(input_df_subset,tmpfile_large, header)
        else:
            input_df_list.append(input_df_subset)
        header = False
        
    if file_is_large:
        process_with_dask(tmpfile_columnfilt=tmpfile_large , outfile_name = outfile_name, config_dict=config_dict)
    else:
        input_df = pd.concat(input_df_list)
        input_reshaped = reshape_input_df(input_df, config_dict)
        input_reshaped.to_csv(outfile_name, sep = "\t", index = None)
    



In [ ]:
#export
import dask.dataframe as dd
import pandas as pd
import glob
import os
import shutil 

def adapt_subtable(input_df_subset, config_dict):
    input_df_subset = filter_input(config_dict.get("filters", {}), input_df_subset)
    if "ion_hierarchy" in config_dict.keys():
        return merge_protein_cols_and_ion_dict(input_df_subset, config_dict)
    else:
        return merge_protein_and_ion_cols(input_df_subset, config_dict)


def reshape_input_df(input_df, config_dict):
    input_df = input_df.astype({'quant_val': 'float'})
    input_reshaped = pd.pivot_table(input_df, index = ['protein', 'ion'], columns = config_dict.get("sample_ID"), values = 'quant_val', fill_value=0)
    if input_reshaped.iloc[:,0].replace(0, np.nan).median() <100: #when values are small, rescale by a constant factor to prevent rounding errors in the subsequent aq analyses
        input_reshaped = input_reshaped *10000

    input_reshaped = input_reshaped.reset_index()
    return input_reshaped


def process_with_dask(*, tmpfile_columnfilt, outfile_name, config_dict):
    df = dd.read_csv(tmpfile_columnfilt, sep = "\t")
    allcols = df[config_dict.get("sample_ID")].drop_duplicates().compute()
    allcols = ['protein', 'ion'] + sorted(allcols)
    df = df.set_index('protein')
    sorted_filedir = f"{tmpfile_columnfilt}_sorted"
    df.to_csv(sorted_filedir, sep = "\t")
    #now the files are sorted and can be pivoted chunkwise (multiindex pivoting at the moment not possible in dask)
    files_dask = glob.glob(f"{sorted_filedir}/*part")
    header = True
    for file in files_dask:
        input_df = pd.read_csv(file, sep = "\t")
        if len(input_df.index) <2:
            continue
        input_reshaped = reshape_input_df(input_df, config_dict)
        input_reshaped = sort_and_add_columns(input_reshaped, allcols)
        write_chunk_to_file(input_reshaped, outfile_name, header)
        header = False
    os.remove(tmpfile_columnfilt)
    shutil.rmtree(sorted_filedir)


def sort_and_add_columns(input_reshaped, allcols):
    missing_cols = set(allcols) - set(input_reshaped.columns)
    input_reshaped[list(missing_cols)] = 0
    input_reshaped = input_reshaped[allcols]
    return input_reshaped





In [ ]:
#export
def reformat_and_write_wideformat_table(peptides_tsv, outfile_name, config_dict):
    input_df = pd.read_csv(peptides_tsv,sep="\t", encoding ='latin1')
    filter_dict = config_dict.get("filters")
    protein_cols = config_dict.get("protein_cols")
    ion_cols = config_dict.get("ion_cols")
    input_df = filter_input(filter_dict, input_df)
    #input_df = merge_protein_and_ion_cols(input_df, config_dict)
    input_df = merge_protein_cols_and_ion_dict(input_df, config_dict)
    if 'quant_prefix' in config_dict.keys():
        quant_prefix = config_dict.get('quant_prefix')
        headers = ['protein', 'ion'] + list(filter(lambda x: x.startswith(quant_prefix), input_df.columns))
        input_df = input_df[headers]
        input_df = input_df.rename(columns = lambda x : x.replace(quant_prefix, ""))

    input_df = input_df.reset_index()
    
    input_df.to_csv(outfile_name, sep = '\t', index = None)

In [ ]:

def read_mq_peptides_table(peptides_tsv, pepheader = "Sequence", protheader = "Leading razor protein"):
    peps = pd.read_csv(peptides_tsv,sep="\t", encoding ='latin1')
    peps = peps[peps["Reverse"] != "+"]
    peps = peps[peps["Potential contaminant"] != "+"]
    if pepheader != None:
        peps = peps.rename(columns = {pepheader : "ion"})
    if protheader != None:
        peps = peps.rename(columns = {protheader: "protein"})
    headers = ['protein', 'ion'] + list(filter(lambda x: x.startswith("Intensity "), peps.columns))
    peps = peps[headers]
    peps = peps.rename(columns = lambda x : x.replace("Intensity ", ""))

    return peps

In [ ]:
#export
from anytree.importer import JsonImporter
import os

def read_condpair_tree(cond1, cond2, results_folder = os.path.join(".", "results")):
    """reads the merged and clustered iontree for a given condpair"""
    condpairname = get_condpairname([cond1, cond2])
    tree_file =os.path.join(results_folder, f"{condpairname}.iontrees.json")
    if not os.path.isfile(tree_file):
        return None
    importer = JsonImporter()
    filehandle = open(tree_file, 'r')
    jsontree = importer.read(filehandle)
    filehandle.close()
    return jsontree

## check for already processed files

In [ ]:
#export
import os
def check_for_processed_runs_in_results_folder(results_folder):
    contained_condpairs = []
    folder_files = os.listdir(results_folder)
    result_files = list(filter(lambda x: "results.tsv" in x ,folder_files))
    for result_file in result_files:
        res_name = result_file.replace(".results.tsv", "")
        if ((f"{res_name}.normed.tsv" in folder_files) & (f"{res_name}.results.ions.tsv" in folder_files)):
            contained_condpairs.append(res_name)
    return contained_condpairs

## Wrapper functions

In [ ]:
#export
import pandas as pd
import os
import pathlib

def import_data(input_file, input_type_to_use = None, samples_subset = None, results_dir = None):
    """
    Function to import peptide level data. Depending on available columns in the provided file,
    the function identifies the type of input used (e.g. Spectronaut, MaxQuant, DIA-NN), reformats if necessary
    and returns a generic wide-format dataframe
    :param file input_file: quantified peptide/ion -level data
    :param file results_folder: the folder where the AlphaQuant outputs are stored
    """

    samples_subset = add_ion_protein_headers_if_applicable(samples_subset)
    if "aq_reformat" in input_file:
        file_to_read = input_file
    else:
        file_to_read = reformat_and_save_input_file(input_file=input_file, input_type_to_use=input_type_to_use)
    
    input_reshaped = pd.read_csv(file_to_read, sep = "\t", encoding = 'latin1', usecols=samples_subset)
    return input_reshaped


def reformat_and_save_input_file(input_file, input_type_to_use = None):
    
    input_type, config_dict_for_type, sep = get_input_type_and_config_dict(input_file, input_type_to_use)
    print(f"using input type {input_type}")
    format = config_dict_for_type.get('format')
    outfile_name = f"{input_file}.{input_type}.aq_reformat.tsv"

    if format == "longtable":
        reformat_and_write_longtable_according_to_config(input_file, outfile_name,config_dict_for_type, sep = sep)
    elif format == "widetable":
        reformat_and_write_wideformat_table(input_file, outfile_name, config_dict_for_type)
    else:
        raise Exception('Format not recognized!')
    return outfile_name




def add_ion_protein_headers_if_applicable(samples_subset):
    if samples_subset is not None:
        return samples_subset + ["ion", "protein"]
    else:
        return None






In [ ]:
#export
import pandas as pd
import os.path
import pathlib

def get_input_type_and_config_dict(input_file, input_type_to_use = None):
    #parse the type of input (e.g. Spectronaut Fragion+MS1Iso) out of the input file


    config_dict = load_config(INTABLE_CONFIG)
    type2relevant_columns = get_type2relevant_cols(config_dict)

    if "aq_reformat.tsv" in input_file:
        input_file = get_original_file_from_aq_reformat(input_file)

    filename = str(input_file)
    if '.csv' in filename:
        sep=','
    if '.tsv' in filename:
        sep='\t'
    if '.txt' in filename:
        sep='\t'

    if 'sep' not in locals():
        raise TypeError(f"neither of the file extensions (.tsv, .csv, .txt) detected for file {input_file}! Your filename has to contain one of these extensions. Please modify your file name accordingly.")



    uploaded_data_columns = set(pd.read_csv(input_file, sep=sep, nrows=1, encoding ='latin1').columns)

    for input_type in type2relevant_columns.keys():
        if (input_type_to_use is not None) and (input_type!=input_type_to_use):
            continue
        relevant_columns = type2relevant_columns.get(input_type)
        relevant_columns = [x for x in relevant_columns if x] #filter None values
        if set(relevant_columns).issubset(uploaded_data_columns):
            config_dict_type =  config_dict.get(input_type)
            return input_type, config_dict_type, sep
    raise TypeError("format not specified in intable_config.yaml!")

import re
def get_original_file_from_aq_reformat(input_file):
    matched = re.match("(.*)(\..*\.)(aq_reformat\.tsv)",input_file)
    return matched.group(1)

In [ ]:
#hide

def test_get_original_file_from_aq_reformat():
    assert get_original_file_from_aq_reformat("yeast_report_fastafiltered.tsv.some.oth.erstuff.spectronaut_fragion_isotopes.aq_reformat.tsv") == "yeast_report_fastafiltered.tsv"

In [ ]:
#export
def import_config_dict():
    config_dict = load_config(INTABLE_CONFIG)
    return config_dict

In [ ]:
#export

import pandas as pd

def load_samplemap(samplemap_file):
    file_ext = os.path.splitext(samplemap_file)[-1]
    if file_ext=='.csv':
        sep=','
    if (file_ext=='.tsv') | (file_ext=='.txt'):
        sep='\t'

    if 'sep' not in locals():
        print(f"neither of the file extensions (.tsv, .csv, .txt) detected for file {samplemap_file}! Trying with tab separation. In the case that it fails, please add the appropriate extension to your file name.")
        sep = "\t"

    return pd.read_csv(samplemap_file, sep = sep, encoding ='latin1', dtype='str')

In [ ]:
#export
def prepare_loaded_tables(data_df, samplemap_df):
    """
    Integrates information from the peptide/ion data and the samplemap, selects the relevant columns and log2 transforms intensities.
    """
    samplemap_df = samplemap_df[samplemap_df["condition"]!=""] #remove rows that have no condition entry
    filtvec_not_in_data = [(x in data_df.columns) for x in samplemap_df["sample"]] #remove samples that are not in the dataframe
    samplemap_df = samplemap_df[filtvec_not_in_data]
    headers = ['protein'] + samplemap_df["sample"].to_list()
    data_df = data_df.set_index("ion")
    for sample in samplemap_df["sample"]:
        data_df[sample] = np.log2(data_df[sample].replace(0, np.nan))
    return data_df[headers], samplemap_df

In [ ]:
#export


#export
class LongTableReformater():
    """Generic class to reformat tabular files in chunks. For the specific cases you can inherit the class and specify reformat and iterate function
    """
    def __init__(self, input_file):
        self._input_file = input_file
        self._reformatting_function = None
        self._iterator_function = self.__initialize_df_iterator__
        self._concat_list = []

    def reformat_and_load_acquisition_data_frame(self):

        input_df_it = self._iterator_function()
        
        input_df_list = []
        for input_df_subset in input_df_it:
            input_df_subset = self._reformatting_function(input_df_subset)
            input_df_list.append(input_df_subset)
        input_df = pd.concat(input_df_list)
        
        return input_df

    def reformat_and_save_acquisition_data_frame(self, output_file):
        
        input_df_it = self._iterator_function()
        write_header = True
        
        for input_df_subset in input_df_it:
            input_df_subset = self._reformatting_function(input_df_subset)
            self.__write_reformatted_df_to_file__(input_df_subset, output_file, write_header)
            write_header = False

    def __initialize_df_iterator__(self):
        return pd.read_csv(self._input_file, sep = "\t", encoding ='latin1', chunksize=1000000)
    
    @staticmethod
    def __write_reformatted_df_to_file__(reformatted_df, filepath ,write_header):
        reformatted_df.to_csv(filepath, header=write_header, mode='a', sep = "\t", index = None)


In [ ]:
#export

import os

class AcquisitionTableHandler():
    def __init__(self, results_dir, samples):
        self._samples = samples
        self._table_infos = AcquisitionTableInfo(results_dir=results_dir)
        self._header_infos = AcquisitionTableHeaders(self._table_infos)
    
    def get_acquisition_info_df(self):
        return self.__get_reformated_df__()

    def save_dataframe_as_new_acquisition_dataframe(self):
        self._output_paths = AcquisitionTableOutputPaths(self._table_infos)
        self.__remove_possible_pre_existing_ml_table__(self._output_paths.output_file_name)
        df_reformater = AcquisitionTableReformater(table_infos = self._table_infos, header_infos=self._header_infos, samples = self._samples, dataframe_already_preformated=False)
        df_reformater.reformat_and_save_acquisition_data_frame(self._output_paths.output_file_name)

    def update_ml_file_location_in_method_parameters_yaml(self):
        method_params = load_method_parameters(self._table_infos._results_dir)
        if self._output_paths == None:
            raise Exception("output paths not initialized! This could be because no dataframe was saved before")
        method_params[self._output_paths.ml_file_accession_in_yaml] = self._output_paths.output_file_name
        save_dict_as_yaml(method_params, self._output_paths.method_parameters_yaml_path)
    
    def __get_reformated_df__(self):
        df_reformater = AcquisitionTableReformater(table_infos = self._table_infos, header_infos=self._header_infos, samples = self._samples, dataframe_already_preformated=True)
        df = df_reformater.reformat_and_load_acquisition_data_frame()
        return df.convert_dtypes()
    
    @staticmethod
    def __remove_possible_pre_existing_ml_table__(output_file_name):
        if os.path.exists(output_file_name):
            os.remove(output_file_name)
            print(f"removed pre existing {output_file_name}")


class AcquisitionTableInfo():
    def __init__(self, results_dir, sep = "\t", decimal = "."):
        self._results_dir = results_dir
        self._sep = sep
        self._decimal = decimal
        self._input_file = self.__get_input_file__()
        self._file_ending_of_formatted_table = ".ml_info_table.tsv"
        self.already_formatted =  self.__check_if_input_file_is_already_formatted__()
        self._input_type, self._config_dict = self.__get_input_type_and_config_dict__()
        self._sample_column = self.__get_sample_column__()
        self.last_ion_level_to_use = self.__get_last_ion_level_to_use__()

    def __get_input_file__(self):
        method_params_dict = load_method_parameters(self._results_dir)
        return method_params_dict.get('ml_input_file')

    def __check_if_input_file_is_already_formatted__(self):
        if self._file_ending_of_formatted_table in self._input_file:
            return True
        else:
            return False

    def __get_input_type_and_config_dict__(self):
        if self.already_formatted:
            original_file = self.__get_location_of_original_file__()
        else:
            original_file = self._input_file
        input_type, config_dict, _ = get_input_type_and_config_dict(original_file)
        return input_type, config_dict
    
    def __get_location_of_original_file__(self):
        return self._input_file.replace(self._file_ending_of_formatted_table, "")
    
    def __get_sample_column__(self):
        return self._config_dict.get("sample_ID")
        
    def __get_last_ion_level_to_use__(self):
        return self._config_dict["ml_level"]


class AcquisitionTableHeaders():
    def __init__(self, acquisition_table_info):

        self._table_info = acquisition_table_info

        self._ion_hierarchy = self.__get_ordered_ion_hierarchy__()
        self._included_levelnames = self.__get_included_levelnames__()
        self._ion_headers_grouped = self.__get_ion_headers_grouped__()
        self._ion_headers = self.__get_ion_headers__()
        self._numeric_headers = self.__get_numeric_headers__()
        self._relevant_headers = self.__get_relevant_headers__()
    
    def __get_ordered_ion_hierarchy__(self):
        ion_hierarchy = self._table_info._config_dict.get("ion_hierarchy")
        hier_key = 'fragion' if 'fragion' in ion_hierarchy.keys() else list(ion_hierarchy.keys())[0]
        ion_hierarchy_on_chosen_key = ion_hierarchy.get(hier_key)
        return ion_hierarchy_on_chosen_key

    def __get_included_levelnames__(self):
        levelnames = self.__get_all_levelnames__(self._ion_hierarchy)
        last_ionlevel_idx = levelnames.index(self._table_info.last_ion_level_to_use)
        return levelnames[:last_ionlevel_idx+1]
    
    @staticmethod
    def __get_all_levelnames__(ion_hierarchy):
        return  ion_hierarchy.get('order')

    def __get_ion_headers_grouped__(self):
        mapping_dict = self.__get_levelname_mapping_dict(self._ion_hierarchy)
        return [mapping_dict.get(x) for x in self._included_levelnames]#on each level there can be multiple names, so it is a list of lists

    @staticmethod
    def __get_levelname_mapping_dict(ion_hierarchy):
        return ion_hierarchy.get('mapping')
    
    def __get_ion_headers__(self):
        return list(itertools.chain(*self._ion_headers_grouped))

    
    def __get_relevant_headers__(self):
        relevant_headers = self._numeric_headers+self._ion_headers + [self._table_info._sample_column]
        return self.__remove_possible_none_values_from_list__(relevant_headers)
    
    @staticmethod
    def __remove_possible_none_values_from_list__(list):
        return [x for x in list if x is not None]

    def __get_numeric_headers__(self):
        df_sample = pd.read_csv(self._table_info._input_file, sep = self._table_info._sep, decimal = self._table_info._decimal, encoding='latin1', nrows=3000) #sample 3000 rows from the df to assess the types of each row
        df_sample = df_sample.replace({False: 0, True: 1})
        numeric_headers =  list(df_sample.select_dtypes(include=np.number).columns)
        numeric_headers = AcquisitionTableHeaderFilter().filter_numeric_headers_if_specified(input_type = self._table_info._input_type, numeric_headers = numeric_headers)
        return numeric_headers


class AcquisitionTableOutputPaths():
    def __init__(self, table_info):
        self._table_info = table_info
        self.output_file_name = self.__get_output_file_name__()
        self.method_parameters_yaml_path = self.__get_method_parameters_yaml_path__()
        self.ml_file_accession_in_yaml = "ml_input_file"

    def __get_output_file_name__(self):
        old_file_name = self._table_info._input_file
        new_file_name = old_file_name+self._table_info._file_ending_of_formatted_table
        return new_file_name

    def __get_method_parameters_yaml_path__(self):
        return f"{self._table_info._results_dir}/aq_parameters.yaml"


class AcquisitionTableReformater(LongTableReformater):
    def __init__(self, table_infos, header_infos, samples, dataframe_already_preformated = False):
        
        LongTableReformater.__init__(self, table_infos._input_file)
        self._table_infos = table_infos
        self._header_infos = header_infos
        self._samples = samples
        self._dataframe_already_preformated = dataframe_already_preformated

        #set the two functions that specify the explicit reformatting
        self._reformatting_function = self.__reformatting_function__
        self._iterator_function = self.__initialize_iterator_with_specified_columns__
    
    def __reformatting_function__(self, input_df_subset):
        input_df_subset = input_df_subset.drop_duplicates()
        input_df_subset = self.__filter_reformated_df_if_necessary__(input_df_subset)
        if not self._dataframe_already_preformated:
            input_df_subset = add_merged_ionnames(input_df_subset, self._header_infos._included_levelnames, self._header_infos._ion_headers_grouped, None, None)
        return input_df_subset

    def __filter_reformated_df_if_necessary__(self, reformatted_df):
        if 'maxquant' in self._table_infos._input_type:
            return reformatted_df
        if 'spectronaut' in self._table_infos._input_type or 'diann' in self._table_infos._input_type:
            return self.__filter_reformatted_dataframe_to_relevant_samples__(reformatted_df)

    def __filter_reformatted_dataframe_to_relevant_samples__(self, input_df_subset):
        return input_df_subset[[x in self._samples for x in input_df_subset[self._table_infos._sample_column]]]
    
    def __initialize_iterator_with_specified_columns__(self):
        cols_to_use = self.__get_cols_to_use__()
        return pd.read_csv(self._table_infos._input_file, sep = self._table_infos._sep, decimal=self._table_infos._decimal, usecols = cols_to_use, encoding ='latin1', chunksize=1000000)

    def __get_cols_to_use__(self):
        cols_to_use = self._header_infos._relevant_headers
        if self._dataframe_already_preformated:
            return cols_to_use+['ion']
        else:
            return cols_to_use




class AcquisitionTableHeaderFilter():
    def __init__(self):
        self._spectronaut_header_filter = lambda x : (("EG." in x) | ("FG." in x)) and ("Global" not in x)
        self._maxquant_header_filter = lambda x : ("Intensity" not in x) and ("Experiment" not in x)

    def filter_numeric_headers_if_specified(self, input_type, numeric_headers):
        if 'spectronaut' in input_type:
            return [x for x in numeric_headers if self._spectronaut_header_filter(x)]
        elif 'maxquant' in input_type:
            return [x for x in numeric_headers if self._maxquant_header_filter(x)]
        else:
            return numeric_headers



    

In [ ]:
#export

def merge_acquisition_df_parameter_df(acquisition_df, parameter_df, groupby_merge_type = 'mean'):
    """acquisition df contains details on the acquisition, parameter df are the parameters derived from the tree
    """
    merged_df = parameter_df.merge(acquisition_df, how = 'left', on = 'ion')
    if groupby_merge_type == 'mean':
        merged_df = merged_df.groupby('ion').mean().reset_index()
    if groupby_merge_type == 'min':
        merged_df = merged_df.groupby('ion').min().reset_index()
    if groupby_merge_type == 'max':
        merged_df = merged_df.groupby('ion').max().reset_index()
    merged_df = merged_df.dropna(axis=1, how='all')
    return merged_df

## Test input parsing

### Compare the in-memory and out-of-memory longformat table loadings

In [ ]:
#hide

import os
import pandas as pd
import shutil
import matplotlib_venn
import alphaquant.diffquant_utils as aqutils

print(os.path.abspath("."))
input_file = "./test_data/unit_tests/input_table_formats/spectronaut.frgions.large.tsv"
outdir = "./test_data/unit_tests/input_table_formats/loading_comparisons"

file_default = "default_out.tsv"
file_dask_proc = "dask_proc_out.tsv"

def test_table_loadings(input_file, outdir, file_default, file_dask_proc):
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        #os.chdir(outdir)

    input_type, config_dict_for_type, sep = aqutils.get_input_type_and_config_dict(input_file)

    aqutils.reformat_and_write_longtable_according_to_config(input_file,outfile_name=file_dask_proc,config_dict=config_dict_for_type, enforce_largefile_processing=True, chunksize=10_000)
    aqutils.reformat_and_write_longtable_according_to_config(input_file,outfile_name=file_default,config_dict=config_dict_for_type, chunksize=10_000)

    df_default = pd.read_csv(file_default, sep = "\t")

    df_dask_proc = pd.read_csv(file_dask_proc, sep = "\t")


    assert df_default.equals(df_dask_proc)

    matplotlib_venn.venn2([set(df_default["ion"]), set(df_dask_proc["ion"])])

    os.remove(file_default)
    os.remove(file_dask_proc)
    shutil.rmtree(outdir)


test_table_loadings(input_file, outdir, file_default, file_dask_proc)

In [ ]:
#hide

import alphaquant.diffquant_utils as aqutils
import alphaquant.diff_analysis_manager as diffmgr
import os
import shutil
outdir = "./test_data/unit_tests/input_table_formats/"

tabledir ="./test_data/unit_tests/input_table_formats/"
results_dir = "./test_data/unit_tests/results/"



input_file = os.path.join(tabledir,"diann.tsv" )
#input_file = os.path.join(tabledir, "spectronaut.tsv")
#input_file = os.path.join(tabledir, "spectronaut_frgion.tsv")
#input_file = os.path.join(tabledir, "mq_peptides.txt")

samplemap_file = os.path.join(tabledir, "samplemap.diann.tsv")
#samplemap_file = os.path.join(tabledir, "samplemap.spectronaut.tsv")
#samplemap_file = os.path.join(tabledir, "samplemap.spectronaut.frgions.tsv")
#samplemap_file = os.path.join(tabledir, "samplemap.mq.tsv")

#import the input table once the input and the results folder are specified. 
# The function automatically recognizes the format (Currently MQ, Spectronaut, DIA-NN configured)
input_data = aqutils.import_data(input_file)
display(input_data)

#get sample names from the imported table
samplenames = aqutils.get_samplenames_from_input_df(input_data)
display(samplenames)

#load the samplemap dataframe (in case the user uploads a file. Basically a pandas import + separator check)
samplemap_df = aqutils.load_samplemap(samplemap_file)
display(samplemap_df)

#compare samplemap and actual table, merge & logtransform intensities
input_processed, samplemap_df_processed = aqutils.prepare_loaded_tables(input_data, samplemap_df)

display(input_processed)
display(samplemap_df_processed)



In [ ]:
#hide
import yaml
import pandas as pd
import numpy as np
from matplotlib_venn import venn2
import matplotlib.pyplot as plt

def compare_generic_table_with_original(preprocessed_input_df, original_input_file, config_yaml,input_typename_config, sep = "\t"):
    id2quant_orig, id2quant_preproc = get_processed_original_id2quant_maps(preprocessed_input_df, original_input_file, config_yaml,input_typename_config)
    keys_orig = set(id2quant_orig.keys())
    keys_preproc = set(id2quant_preproc.keys())
    keydiff = keys_preproc.difference(keys_orig)
    keys_orig = sorted(keys_orig)
    keys_preproc = sorted(keys_preproc)
    print(list(keys_orig)[:10])
    print(list(keys_preproc)[:10])
    
    
    assert(len(keydiff)==0) #check that all keys in the preprocessed set are part of the original set

    venn2([set(id2quant_orig.keys()), set(id2quant_preproc.keys())], ["original", "preprocessed"])
    
    quantvec_orig = np.array([id2quant_orig.get(x)for x in id2quant_preproc.keys()])
    quantvec_preproc = np.array([id2quant_preproc.get(x)for x in id2quant_preproc.keys()])
    unequal_quant = [id2quant_orig.get(x)!=id2quant_preproc.get(x) for x in id2quant_preproc.keys()]
    unequal_quant_scaled = [id2quant_orig.get(x)*10000!=id2quant_preproc.get(x) for x in id2quant_preproc.keys()]
    print(sum(unequal_quant))
    print(sum(unequal_quant_scaled))
    plt.show()
    plt.scatter(quantvec_orig, quantvec_preproc)
    plt.show()
    corrcoeff = np.corrcoef(quantvec_orig,quantvec_preproc)[0][1]
    print(f"correlation between both processings: {corrcoeff}")
    assert(corrcoeff>0.999)



def get_processed_original_id2quant_maps(preprocessed_input_df, original_input_file, config_yaml,input_typename_config, sep = "\t"):
    config_all = yaml.safe_load(open(config_yaml, 'r'))
    config_dict = config_all.get(input_typename_config)
    id_cols = config_dict.get("ion_cols") + [config_dict.get("sample_ID")]
    quant_col = list(config_dict.get("quant_ID").values())
    id2quant_orig = get_id2quant_original(original_input_file, id_cols, quant_col, sep)
    id2quant_preproc = get_id2quant_processed(preprocessed_input_df, id_cols, quant_col)
    return id2quant_orig, id2quant_preproc


def get_id2quant_original(original_input_file, id_cols, quant_col, sep):
    print(id_cols)
    print(id_cols+quant_col)
    orig_df = pd.read_csv(original_input_file, sep=sep, usecols= id_cols+quant_col)
    orig_df["compareID"] = orig_df[id_cols].astype('string').sum(axis = 1)
    display(orig_df)
    id2quant = dict(zip(orig_df["compareID"], orig_df[quant_col[0]]))
    id2quant = {k: round(v,3) for k, v in id2quant.items()}
    return id2quant


def get_id2quant_processed(preprocessed_input_df, id_cols, quant_col):
    compare_IDs = []
    quantvals = []
    for column in preprocessed_input_df.columns:
        if(column == "protein"):
            continue
        id = pd.Series([column for x in range(len(preprocessed_input_df.index))]).to_numpy()[0]
        reformated_pep_id = [x.split("_MOD_")[1].replace("_CHARGE_", "")[:-1]+id for x in preprocessed_input_df.index]
        compare_IDs.extend(reformated_pep_id)

        quantvals.extend(list(preprocessed_input_df[column]))
    
    id2quant = dict(zip(compare_IDs, quantvals))
    id2quant = {k: round(2**v,3) for k, v in id2quant.items() if ~np.isnan(v)}
    return id2quant


import alphaquant.diffquant_utils as aqutils
import os


input_file = os.path.join(tabledir, "spectronaut.tsv")
samplemap_file = os.path.join(tabledir, "samplemap.spectronaut.tsv")


input_data = aqutils.import_data(input_file)
samplemap_df = aqutils.load_samplemap(samplemap_file)
input_processed, samplemap_df_processed = aqutils.prepare_loaded_tables(input_data, samplemap_df)
compare_generic_table_with_original(input_processed, input_file, os.path.join("alphaquant","configs","intable_config.yaml"), "spectronaut_precursor_v2")
